In [ ]:
import os
for key, value in os.environ.items():
    print(f"{key}: {value}")

In [2]:
from scripts.score import main

In [ ]:
main(
    input_filename="data/prompts.jsonl",
    column_name="prompt",
    output_folder="data/scores",
)